In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd


In [30]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model

In [45]:
#load data and convert from integer to float
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [46]:
#use tf.data to create batch size and shuffle data
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [47]:
#Build a model using Keras model Subclassing API
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

In [34]:
#Choose a optimizer and loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [35]:
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

#Select metrics to measure the loss and the accuracy of the model   

In [53]:
#train model using tf.GradientTape
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [54]:
#Test model
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [55]:
epochs = 5

for epoch in range(epochs):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  output = "Epoch: {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test_Accuracy: {}"
  print(output.format(
      epoch+1,
      train_loss.result(),
      train_accuracy.result() * 100,
      test_loss.result(),
      test_accuracy.result() * 100
  ))


Epoch: 1, Loss: 0.021910525858402252, Accuracy: 99.30333709716797, Test Loss: 0.05030765011906624, Test_Accuracy: 98.43999481201172
Epoch: 2, Loss: 0.013101078569889069, Accuracy: 99.56832885742188, Test Loss: 0.059232886880636215, Test_Accuracy: 98.22999572753906
Epoch: 3, Loss: 0.009463250637054443, Accuracy: 99.68167114257812, Test Loss: 0.07148577272891998, Test_Accuracy: 98.2699966430664
Epoch: 4, Loss: 0.0061687589623034, Accuracy: 99.78333282470703, Test Loss: 0.05767173320055008, Test_Accuracy: 98.54000091552734
Epoch: 5, Loss: 0.005504497326910496, Accuracy: 99.82833099365234, Test Loss: 0.07544629275798798, Test_Accuracy: 98.38999938964844
